In [2]:
experiment=None

In [1]:
#Imports
from comet_ml import Experiment
experiment = Experiment(api_key="YkPEmantOag1R1VOJmXz11hmt", parse_args=False, project_name='SegNet')

COMET INFO: old comet version (1.0.44) detected. current: 1.0.53 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Failing to collect the installed os packages
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yikeqicn/segnet/f198089f14974221b181056ef537ac62



In [10]:
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse
from glob import glob
import cv2
import numpy as np
import os
import time

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import IRS #ArtPrintNoIntsectLBW,ArtPrintNoIntsectLBW_biameyd_siameyd,ArtPrintNoIntsectLBW_bpr_spr,ArtPrintNoIntsectLBW_biameyd_sprt
from Model_Unet_github import *

home = os.environ['HOME']

In [3]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
parser.add_argument("-name", default='segnet_unet_lbw_all_combine_100_epoches', type=str, help="name of the log") #debug model_intersect # segnet_no_intersect_1conv_64_channels_30epoch_unet_github
parser.add_argument("-gpu", default='1', type=str, help="gpu numbers")

parser.add_argument("-train", default=False, help="train the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=True, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=100, type=int,help='max epoch numbers')



# file paths
parser.add_argument('-ckpt_root', default="/root/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/root/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()
args = parser.parse_known_args()[0]

name = args.name

experiment.set_name('segnet_unet_lbw_all_test_irs_handwriting')
experiment.log_parameters(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath = join(ckptroot, name)
if args.name=='debug': shutil.rmtree(args.ckptpath, ignore_errors=True)
os.makedirs(args.ckptpath, exist_ok=True)


In [4]:
def checkArgs():
    if args.test:
        print('The model is set to Testing')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid testing dir: %s"%FLAGS.test_dir)
    elif args.transfer:
        print('The model is set to transfer learn from ckpt')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Image dir: %s"%FLAGS.image_dir)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)
    else:
        print('The model is set to Training')
        print("Max training Iteration: %d"%args.max_epoch)
        print("Initial lr: %f"%args.lrInit)
        print("First Drop Steps: %i"%args.lrDrop1)
        print("Second Drop Steps: %i"%args.lrDrop2)
        print("Data root: %s"%args.data_root)
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)

    print("Batch Size: %d"%args.batch_size)

In [5]:
checkArgs()

The model is set to Testing
Check point file: /root/ckpt/segnet_unet_lbw_all_combine_100_epoches
Batch Size: 10


In [6]:
#model=Model(args, experiment, loss_weight=(0.5,0.5), mustRestore=True)

# Large Test Set

In [16]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        #arprint=ArtPrint(args.data_root, transform=transform_train)
        arhh=ArtPrintNoIntsectLBW_biameyd_siameyd(args.data_root, transform=transform_train)
        arph=ArtPrintNoIntsectLBW(args.data_root, transform=transform_train)
        arpp=ArtPrintNoIntsectLBW_bpr_spr(args.data_root, transform=transform_train)
        arhp=ArtPrintNoIntsectLBW_biameyd_sprt(args.data_root, transform=transform_train)
        concat=ConcatDataset([arhh,arph,arpp,arhp])
        print(len(concat))
        idxTrain = int( len(concat)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        #return trainset, concat
        #weight gen
        # modified here:
        samples_all=[]
        print(trainset.dataset.datasets)
        for ds in trainset.dataset.datasets:
            samples_all+=ds.samples
        #samples_all=trainset.dataset.samples
        print(len(samples_all))
        print(samples_all[0])
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),samples_all))/(args.image_h*args.image_w*len(samples_all))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    elif args.test:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        irs = IRS(args.data_root,transform=transform_train) #yike todo
        testloader = DataLoader(irs, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        model=Model(args, experiment, loss_weight=[0.5,0.5], mustRestore=False)
        for idx, (images, labels) in enumerate(testloader):
            images=images.numpy()
            model.imageClean(images)
    else:
        
        pass # for now, leave it pass 

In [17]:
main()

The model is set to Testing
Check point file: /root/ckpt/segnet_unet_lbw_all_combine_100_epoches
Batch Size: 10
/root/datasets/irs_handwriting already exists, skipping download
GGG
[None, 32, 128, 1]
0 conv1: (?, ?, ?, 32)
0 conv2: (?, ?, ?, 32)
1 conv1: (?, ?, ?, 64)
1 conv2: (?, ?, ?, 64)
2 conv1: (?, ?, ?, 128)
2 conv2: (?, ?, ?, 128)
1 h_deconv: (?, ?, ?, 64)
1 h_deconv_concat: (?, ?, ?, ?)
1 h_conv1_post_deconv: (?, ?, ?, 64)
1 h_conv2_post_deconv: (?, ?, ?, 64)
0 h_deconv: (?, ?, ?, 32)
0 h_deconv_concat: (?, ?, ?, ?)
0 h_conv1_post_deconv: (?, ?, ?, 32)
0 h_conv2_post_deconv: (?, ?, ?, 32)
0 outmap: (?, ?, ?, 2)
(?, ?, ?, 2)
loss: ()
INFO:tensorflow:Summary name loss/cross_entropy (raw) is illegal; using loss/cross_entropy__raw_ instead.


INFO:tensorflow:Summary name loss/cross_entropy (raw) is illegal; using loss/cross_entropy__raw_ instead.


INFO:tensorflow:Summary name loss/total_loss (raw) is illegal; using loss/total_loss__raw_ instead.


INFO:tensorflow:Summary name loss/total_loss (raw) is illegal; using loss/total_loss__raw_ instead.
COMET ERROR: Failed to extract parameters from Estimator.init()


toto_loss_shape: Tensor("loss/total_loss:0", shape=(), dtype=float32)
Python: 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
Tensorflow: 1.12.0-rc0
INFO:tensorflow:Restoring parameters from /root/ckpt/segnet_unet_lbw_all_combine_100_epoches/model-80


INFO:tensorflow:Restoring parameters from /root/ckpt/segnet_unet_lbw_all_combine_100_epoches/model-80


Init with stored values from /root/ckpt/segnet_unet_lbw_all_combine_100_epoches/model-80


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


# Small Test Set

In [44]:
paths=glob(join(args.data_root,'irs_handwriting','**/**/**jpg'))
paths_2=glob(join(args.data_root,'small_test_dataset','**jpg'))+glob(join(args.data_root,'small_test_dataset','**png'))
#path=paths[15]
#img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
#img=cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)
#img=np.expand_dims(img,3)
imgs=[ np.expand_dims(cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC),3) for path in paths_2]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


In [45]:
model.imageClean(imgs)

In [42]:
from __future__ import print_function, division, absolute_import, unicode_literals
import tensorflow as tf

import cv2

import os, sys
import numpy as np
import math
from datetime import datetime
import time
from PIL import Image
from math import ceil
from collections import OrderedDict
import logging
'''
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import gen_nn_ops
'''
# modules

from utils import get_image_summary,log_images,_variable_with_weight_decay, _variable_on_cpu, _add_loss_summaries, _activation_summary, print_hist_summery, get_hist, per_class_acc, writeImage

#from datasets import ArtPrint
from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
from os.path import join, basename, dirname
#from Inputs import *

# model layers
def weight_variable(shape, stddev=0.1, name="weight"):
    shape=np.array(shape)
    #print(shape)
    #print(stddev)
    initial = tf.truncated_normal(shape, stddev=stddev)
    return tf.Variable(initial, name=name)

def weight_variable_devonc(shape, stddev=0.1, name="weight_devonc"):
    shape=np.array(shape)
    return tf.Variable(tf.truncated_normal(shape, stddev=stddev), name=name)

def bias_variable(shape, name="bias"):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

def conv2d(x, W, b, keep_prob_):
    with tf.name_scope("conv2d"):
        conv_2d = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')#'VALID'
        conv_2d_b = tf.nn.bias_add(conv_2d, b)
        return tf.nn.dropout(conv_2d_b, keep_prob_)

def deconv2d(x, W,stride):
    with tf.name_scope("deconv2d"):
        x_shape = tf.shape(x)
        output_shape = tf.stack([x_shape[0], x_shape[1]*2, x_shape[2]*2, x_shape[3]//2])
        return tf.nn.conv2d_transpose(x, W, output_shape, strides=[1, stride, stride, 1], padding='SAME', name="conv2d_transpose") #'VALID'

def max_pool(x,n):
    return tf.nn.max_pool(x, ksize=[1, n, n, 1], strides=[1, n, n, 1], padding='SAME') #'VALID'

def crop_and_concat(x1,x2):
    with tf.name_scope("crop_and_concat"):
        x1_shape = tf.shape(x1)
        x2_shape = tf.shape(x2)
        # offsets for the top left corner of the crop
        offsets = [0, (x1_shape[1] - x2_shape[1]) // 2, (x1_shape[2] - x2_shape[2]) // 2, 0]
        size = [-1, x2_shape[1], x2_shape[2], -1]
        x1_crop = tf.slice(x1, offsets, size)
        return tf.concat([x1_crop, x2], 3)

def pixel_wise_softmax(output_map):
    with tf.name_scope("pixel_wise_softmax"):
        max_axis = tf.reduce_max(output_map, axis=3, keepdims=True)
        exponential_map = tf.exp(output_map - max_axis)
        normalize = tf.reduce_sum(exponential_map, axis=3, keepdims=True)
        return exponential_map / normalize

def cross_entropy(y_,output_map):
    return -tf.reduce_mean(y_*tf.log(tf.clip_by_value(output_map,1e-10,1.0)), name="cross_entropy")

#unet setting
def create_conv_net(x, keep_prob, channels, n_class, layers=3, features_root=16, filter_size=3, pool_size=2,
                    summaries=True):
    """
    Creates a new convolutional unet for the given parametrization.
    :param x: input tensor, shape [?,nx,ny,channels]
    :param keep_prob: dropout probability tensor
    :param channels: number of channels in the input image
    :param n_class: number of output labels
    :param layers: number of layers in the net
    :param features_root: number of features in the first layer
    :param filter_size: size of the convolution filter
    :param pool_size: size of the max pooling operation
    :param summaries: Flag if summaries should be created
    """

    logging.info(
        "Layers {layers}, features {features}, filter size {filter_size}x{filter_size}, pool size: {pool_size}x{pool_size}".format(
            layers=layers,
            features=features_root,
            filter_size=filter_size,
            pool_size=pool_size))

    # Placeholder for the input image
    with tf.name_scope("preprocessing"):
        nx = tf.shape(x)[1]
        ny = tf.shape(x)[2]
        #nx=32
        #ny=128
        #channels=1
        x_image = tf.reshape(x, tf.stack([-1, nx, ny, channels]))
        in_node = x_image
        batch_size = tf.shape(x_image)[0]

    weights = []
    biases = []
    convs = []
    pools = OrderedDict()
    deconv = OrderedDict()
    dw_h_convs = OrderedDict()
    up_h_convs = OrderedDict()

    in_size = 1000  #?????????????????????
    size = in_size
    # down layers
    for layer in range(0, layers):
        with tf.name_scope("down_conv_{}".format(str(layer))):
            features = 2 ** layer * features_root
            stddev = np.sqrt(2 / (filter_size ** 2 * features))
            if layer == 0:
                w1 = weight_variable([filter_size, filter_size, channels, features], stddev, name="w1")
            else:
                w1 = weight_variable([filter_size, filter_size, features // 2, features], stddev, name="w1")

            w2 = weight_variable([filter_size, filter_size, features, features], stddev, name="w2")
            b1 = bias_variable([features], name="b1")
            b2 = bias_variable([features], name="b2")

            conv1 = conv2d(in_node, w1, b1, keep_prob)
            print(str(layer)+' conv1: '+str(conv1.get_shape()))
            tmp_h_conv = tf.nn.relu(conv1)
            conv2 = conv2d(tmp_h_conv, w2, b2, keep_prob)
            print(str(layer)+' conv2: '+str(conv2.get_shape()))
            dw_h_convs[layer] = tf.nn.relu(conv2)

            weights.append((w1, w2))
            biases.append((b1, b2))
            convs.append((conv1, conv2))

            size -= 2 * 2 * (filter_size // 2) # valid conv
            if layer < layers - 1:
                pools[layer] = max_pool(dw_h_convs[layer], pool_size)
                in_node = pools[layer]
                size /= pool_size

    in_node = dw_h_convs[layers - 1]

    # up layers
    for layer in range(layers - 2, -1, -1):
        with tf.name_scope("up_conv_{}".format(str(layer))):
            features = 2 ** (layer + 1) * features_root
            stddev = np.sqrt(2 / (filter_size ** 2 * features))

            wd = weight_variable_devonc([pool_size, pool_size, features // 2, features], stddev, name="wd")
            bd = bias_variable([features // 2], name="bd")
            h_deconv = tf.nn.relu(deconv2d(in_node, wd, pool_size) + bd)
            print(str(layer)+' h_deconv: '+str(h_deconv.get_shape()))
            h_deconv_concat = crop_and_concat(dw_h_convs[layer], h_deconv)
            print(str(layer)+' h_deconv_concat: '+str(h_deconv_concat.get_shape()))
            deconv[layer] = h_deconv_concat

            w1 = weight_variable([filter_size, filter_size, features, features // 2], stddev, name="w1")
            w2 = weight_variable([filter_size, filter_size, features // 2, features // 2], stddev, name="w2")
            b1 = bias_variable([features // 2], name="b1")
            b2 = bias_variable([features // 2], name="b2")

            conv1 = conv2d(h_deconv_concat, w1, b1, keep_prob)
            h_conv = tf.nn.relu(conv1)
            print(str(layer)+' h_conv1_post_deconv: '+str(h_conv.get_shape()))
            conv2 = conv2d(h_conv, w2, b2, keep_prob)
            in_node = tf.nn.relu(conv2)
            up_h_convs[layer] = in_node
            print(str(layer)+' h_conv2_post_deconv: '+str(in_node.get_shape()))

            weights.append((w1, w2))
            biases.append((b1, b2))
            convs.append((conv1, conv2))

            size *= pool_size
            size -= 2 * 2 * (filter_size // 2) # valid conv

    # Output Map
    with tf.name_scope("output_map"):
        weight = weight_variable([1, 1, features_root, n_class], stddev)
        bias = bias_variable([n_class], name="bias")
        conv = conv2d(in_node, weight, bias, tf.constant(1.0))
        print(str(layer)+' outmap: '+str(conv.get_shape()))
        
        #output_map = tf.nn.relu(conv)
        output_map=conv # no activation, to be consistant with other models and leverage previous loss/prediction structures yike !!!!
        up_h_convs["out"] = output_map

    if summaries:
        with tf.name_scope("summaries"):
            for i, (c1, c2) in enumerate(convs):
                tf.summary.image('summary_conv_%02d_01' % i, get_image_summary(c1))
                tf.summary.image('summary_conv_%02d_02' % i, get_image_summary(c2))

            for k in pools.keys():
                tf.summary.image('summary_pool_%02d' % k, get_image_summary(pools[k]))

            for k in deconv.keys():
                tf.summary.image('summary_deconv_concat_%02d' % k, get_image_summary(deconv[k]))

            for k in dw_h_convs.keys():
                tf.summary.histogram("dw_convolution_%02d" % k + '/activations', dw_h_convs[k])

            for k in up_h_convs.keys():
                tf.summary.histogram("up_convolution_%s" % k + '/activations', up_h_convs[k])

    variables = []
    for w1, w2 in weights:
        variables.append(w1)
        variables.append(w2)

    for b1, b2 in biases:
        variables.append(b1)
        variables.append(b2)

    return output_map, variables, int(in_size - size)

### model ###


class Model:
    # Constants describing the training process. probably move to hyperprameters at main beginning
    MOVING_AVERAGE_DECAY = 0.9999     # The decay to use for the moving average.
    NUM_EPOCHS_PER_DECAY = 350.0      # Epochs after which learning rate decays.
    LEARNING_RATE_DECAY_FACTOR = 0.1  # Learning rate decay factor.
    
    def __init__(self, args, experiment, loss_weight, mustRestore=False):
      "init model: SegNet model"
      self.args = args
      self.experiment=experiment     
      self.mustRestore = mustRestore
      
      ###model hyperparameters###
      self.num_classes=args.num_class  
      # self.FilePaths = FilePaths
      self.batch_size = args.batch_size
      self.lrInit = args.lrInit
      self.loss_weight=loss_weight
      
      ###input### -- try to only set up graph once, combine train and test, by yike
      tf.reset_default_graph() # yike reset default graph
      self.input_images= tf.placeholder( tf.float32, shape=[None, self.args.image_h, self.args.image_w, self.args.image_c]) # try my best to make runtime batch_size flexible
      self.input_labels= tf.placeholder(tf.int64, shape=[None, self.args.image_h, self.args.image_w, 1])
      self.phase_train= tf.placeholder(tf.bool, name='phase_train')
      self.global_step=tf.Variable(0,trainable=False)
      self.learning_rate=tf.placeholder(tf.float32, shape=[])
    
      ###graph### -- combine 
      self.logit= self.setup_graph(self.input_images, self.phase_train)
      self.loss=self.cal_loss(self.logit,self.input_labels) 
      self.pred=tf.argmax(self.logit,axis=3)
      self.train_op=self.train(self.loss,self.learning_rate,self.global_step)

      
      ###session and saver###
      #self.saver=tf.train.Saver(max_to_keep=1)
      #self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, gpu_options=tf.GPUOptions(allow_growth=True)))
      (self.sess,self.saver) =self.initTF()
    
    ### 1. loss factory ###
    def weighted_loss(self, logits, labels): # num_classes, head=None):
      """ median-frequency re-weighting """
      with tf.name_scope('loss'):
           #print('w_llll')
           logits = tf.reshape(logits, (-1, self.num_classes))
           #print(logits.get_shape())
           epsilon = tf.constant(value=1e-10)

           logits = logits + epsilon

           # consturct one-hot label array
           label_flat = tf.reshape(labels, (-1, 1))
           #print(label_flat.get_shape())

           # should be [batch ,num_classes]
           labels = tf.reshape(tf.one_hot(label_flat, depth=self.num_classes), (-1, self.num_classes))
           # print(labels.get_shape())

           softmax = tf.nn.softmax(logits)
           #print(softmax.get_shape())
#        print(epsilon.get_shape())

#        print((labels * tf.log(softmax + epsilon)).get_shape())
#        print(head.shape)
#        print(tf.multiply(labels * tf.log(softmax + epsilon), head))
        
           cross_entropy = -tf.reduce_sum(tf.multiply(labels * tf.log(softmax + epsilon), self.loss_weight), axis=[1])
#        print(cross_entropy.get_shape()) # yike head -> self.loss_weight

           cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
#        print(cross_entropy_mean.get_shape())
           tf.add_to_collection('losses', cross_entropy_mean)
           
           loss = tf.add_n(tf.get_collection('losses'), name='total_loss')
           print('loss: '+str(loss.get_shape()))      

      return loss
    
    def cal_loss(self,logits,labels):
       labels = tf.cast(labels, tf.int32)
       return self.weighted_loss(logits, labels)
    #self.weighted_loss(logits, labels, num_classes=NUM_CLASSES, head=loss_weight)
    
    ###2. train optimizer factory ###    
    def train(self,total_loss, lr, global_step):
       # all of them are tensor
       #total_sample = 274 yike: ok to comment out?
       #num_batches_per_epoch = 274/1 yike: ok to comment out?

       loss_averages_op = _add_loss_summaries(total_loss)
       # Compute gradients.
       with tf.control_dependencies([loss_averages_op]):
         #print('try...')
         opt = tf.train.AdamOptimizer(lr)
         print('toto_loss_shape: '+str(total_loss))
         opt.compute_gradients(total_loss)
         grads = opt.compute_gradients(total_loss)
         #print(grads)
         apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)

         # Add histograms for trainable variables.
         for var in tf.trainable_variables():
           tf.summary.histogram(var.op.name, var)

         # Add histograms for gradients.
         for grad, var in grads:
           if grad is not None:
             tf.summary.histogram(var.op.name + '/gradients', grad)

         # Track the moving averages of all trainable variables.
         variable_averages = tf.train.ExponentialMovingAverage(Model.MOVING_AVERAGE_DECAY, global_step)
         variables_averages_op = variable_averages.apply(tf.trainable_variables())

         with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
           train_op = tf.no_op(name='train')

       return train_op
    
    
    ###3. graph factory ###
    
    def setup_graph(self, images, phase_train): # previous inference() labels,inference, batch_size -- in order to get batch_size at running time 
       #rather than using fixed batch_size in graph set up, revise it in inference:
       #batchsize=tf.shape(images)[0] # yike !!!
       print('GGG')
       input_shape=images.get_shape().as_list()
       print(input_shape)

#       create_conv_net(x, keep_prob, channels, n_class, layers=3, features_root=16, filter_size=3, pool_size=2,
#                    summaries=True) 
        
       logit,_,__=create_conv_net(x=images, keep_prob=0.8, channels=input_shape[3], n_class=self.num_classes, layers=3, features_root=32, filter_size=3) 
       print(logit.get_shape()) 
       ''' 
       """ Start Classify """
       
       # output predicted class number (6)
       with tf.variable_scope('conv_classifier') as scope:
         kernel = _variable_with_weight_decay('weights',
                                           shape=[1, 1, 64, self.num_classes],
                                           initializer=msra_initializer(1, 64),
                                           wd=0.0005)
         conv = tf.nn.conv2d(conv_decode1, kernel, [1, 1, 1, 1], padding='SAME')
         print('cv')
         print(conv.get_shape())
         biases = _variable_on_cpu('biases', [self.num_classes], tf.constant_initializer(0.0))
         print(biases.get_shape())
         logit= tf.nn.bias_add(conv, biases, name=scope.name)
         #conv_classifier = tf.nn.bias_add(conv, biases, name=scope.name)
         #print(conv_classifier.get_shape())
         #logit = conv_classifier
         #print('LLL')
         #print(labels)
         #print(conv_classifier)
    
         #loss = cal_loss(conv_classifier, labels)
         print(logit.get_shape())
         '''
       return logit # loss

    ###4. initialization###
    def initTF(self):
       "initialize TF"
       print('Python: ' + sys.version)
       print('Tensorflow: ' + tf.__version__)

       sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, gpu_options=tf.GPUOptions(allow_growth=True)))
       saver = tf.train.Saver(max_to_keep=1)  # saver saves model to file

       # Restore from saved model in current checkpoint directory
       latestSnapshot = tf.train.latest_checkpoint(self.args.ckptpath)  # is there a saved model?
       if self.mustRestore and not latestSnapshot: # if model must be restored (for inference), there must be a snapshot
         raise Exception('No saved model found in: ' + self.args.ckptpath)

       if latestSnapshot: # load saved model if available
         saver.restore(sess, latestSnapshot)
         print('Init with stored values from ' + latestSnapshot)
       else:
         sess.run(tf.global_variables_initializer())
         print('Ran global_variables_initializer')

         # initialize params from other model (transfer learning)
       if self.args.transfer:
         utils.maybe_download(source_url=self.args.urlTransferFrom,
                         filename=join(self.args.ckptpath, 'transferFrom'),
                         target_directory=None,
                         filetype='folder',
                         force=True)
         saverTransfer = tf.train.Saver(tf.trainable_variables()[:-1])  # load all variables except from logit (classification) layer
         saverTransfer.restore(sess, glob(join(self.args.ckptpath, 'transferFrom', 'model*'))[0].split('.')[0])
         print('Loaded variable values (except logit layer) from ' + self.args.urlTransferFrom)

       return (sess, saver) 
    
    ###5. training ###
    def trainBatch(self, images, labels):
       "feed a batch into the NN to train it"
    
       #sparse = self.toSparse(labels)
       #lrnrate = self.lrInit if self.batchesTrained < self.args.lrDrop1 else (
       #self.lrInit*1e-1 if self.batchesTrained < self.args.lrDrop2 else self.lrInit*1e-2)  # decay learning rate
       train_step=self.global_step.eval(session=self.sess)
       """ fix lr """ ## To Ronny, change the schedule?
       #lr = self.lrInit
       lr = self.lrInit if train_step < self.args.lrDrop1 else (
            self.lrInit*1e-1 if train_step < self.args.lrDrop2 else self.lrInit*1e-2) #yike
       (_, lossVal) = self.sess.run([self.train_op, self.loss],
                                  {self.input_images: images,
                                   self.input_labels: labels,
                                   self.learning_rate: lr, 
                                   self.phase_train: True})
       #self.batchesTrained += 1
       return lossVal
    
    def training(self, loader, validateloader=None,testloader=None):
       "train NN"
       epoch = 0  # number of training epochs since start
       best_accuracy=0.0
       step = 0
       while True:
         epoch += 1; print('Epoch:', epoch, ' Training...')
         # train
         counter = 0
         #step = 0 
         for idx, (images, labels) in enumerate(loader):
            images=images.numpy()
            labels=labels.numpy()
            loss_value=self.trainBatch(images,labels)
            assert not np.isnan(loss_value), 'Model diverged with loss = NaN'
            step+=1
            
            if idx % 100 ==0:
              print('TRAIN: Batch:', idx/len(loader), 'Loss:', loss_value)
              self.experiment.log_metric('train/loss', loss_value, step)
              logits=self.sess.run(self.logit, feed_dict={self.input_images: images, # check in, comment out in formal run
                                   #self.input_labels: labels,
                                   #self.learning_rate: lr, 
                                   self.phase_train: False})
              train_acc,train_acc_classes=per_class_acc(logits,labels)  # check in, comment out in formal run

         # train log:
         self.experiment.log_metric('train/acc',train_acc,step)
         self.experiment.log_metric('train/cap_0',train_acc_classes[0],step)
         self.experiment.log_metric('train/cap_1',train_acc_classes[1],step)
                
         #validate:
         if validateloader !=None: 
           avg_batch_loss,acc_total,cap_0,cap_1=self.validate(validateloader,epoch)
         else:
           avg_batch_loss,acc_total,cap_0,cap_1=self.validate(loader,epoch)
         self.experiment.log_metric('valid/acc',acc_total,step)
         self.experiment.log_metric('valid/cap_0',cap_0,step)
         self.experiment.log_metric('valid/cap_1',cap_1,step)
         self.experiment.log_metric('valid/loss',avg_batch_loss,step)   
         

         #test:
         if testloader !=None:
           acc_total,cap_0,cap_1=self.validate(testloader,epoch,is_testing=True)
           self.experiment.log_metric('test/acc',acc_total,step)
           self.experiment.log_metric('test/cap_0',cap_0,step)
           self.experiment.log_metric('test/cap_1',cap_1,step)
           
         # log best metrics
         if acc_total > best_accuracy: # if best validation accuracy so far, save model parameters
           print('Character error rate improved, save model')
           best_accuracy = acc_total
           noImprovementSince = 0
           self.save(epoch)
           open(join(self.args.ckptpath, 'accuracy.txt'), 'w').write('Validation accuracy, class 0, class 1 capture rates of saved model: %f%%, %f%% and %f%% ' % (acc_total * 100.0, cap_0 * 100.0, cap_1 * 100.0))
           self.experiment.log_metric('best/acc',acc_total,step)
           self.experiment.log_metric('best/cap_0',cap_0,step)
           self.experiment.log_metric('best/cap_1',cap_1,step)         
         else:
           print('Character error rate not improved')
           noImprovementSince += 1

         # stop training
         if epoch>=self.args.max_epoch: print('Done with training at epoch', epoch, 'sigoptObservation='+str(best_accuracy)); break            
            
            
    ###6. testing / validate ###
    def validate(self, loader, epoch, is_testing=False):
       "validate NN"
       if not is_testing: print('Validating NN')
       else: print('Testing NN')
       total_val_loss = 0.0
       #num_batches=len(loader)
       hist= np.zeros((self.num_classes, self.num_classes))
        
       image_upload_count=0 
       for idx, (images, labels) in enumerate(loader):
         images=images.numpy()
         labels=labels.numpy()
         val_loss,val_logit=self.sess.run([self.loss,self.logit],feed_dict=
                                {self.input_images: images, # check in, comment out in formal run
                                self.input_labels: labels,
                                self.phase_train: False}) #self.loss,val_loss,
          
         total_val_loss+=val_loss
         hist+=get_hist(val_logit,labels)
       #val_loss=total_val_loss / len(validateloader)*batch_size

         if epoch==self.args.max_epoch and image_upload_count<1000: # decide how many images to upload
            pred=val_logit.argmax(3)
            images=np.squeeze(images,axis=3)
            image_upload_count=log_images(images,pred,image_upload_count,self.experiment,self.args.ckptpath)
            
       avg_batch_loss=total_val_loss/idx     
       cls_sample_nums=hist.sum(1).astype(float)
       capture_array=np.diag(hist)
       acc_total = capture_array.sum() / hist.sum()
       capture_rate_ls=[]
       for cls in range(self.num_classes):
         if cls_sample_nums[cls]==0:
            capture_rate=0.0
         else:
            capture_rate=capture_array[cls]/cls_sample_nums[cls]
         capture_rate_ls.append(capture_rate)
       #iu = np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))
       #mean_iu=np.nanmean(iu)
       print('VALID: Total accuracy: %f%%. Class 0 capture: %f%%. Class 1 capture: %f%%' % (acc_total * 100.0, capture_rate_ls[0] * 100.0, capture_rate_ls[1]*100.0)) 
       return avg_batch_loss,acc_total,capture_rate_ls[0],capture_rate_ls[1]
    
    ###7. infer ###
    def inferBatch(self, imgs): # modify to compatible to torch. previous def inferBatch(self, batch)
       "feed a batch into the NN to recngnize the texts"
  
       bt_size=len(imgs) # yike !!!!!!!!

       pred = self.sess.run(self.pred,feed_dict=
                                {self.input_images: imgs, # check in, comment out in formal run
                                #self.input_labels: labels,
                                self.phase_train: False}) #yike self.batchsize!!!!!!!!!
       return pred
    def imageClean(self,imgs):
       pred=self.inferBatch(imgs)
       bt_size=len(imgs)
       images=np.squeeze(imgs,axis=3)
       image_upload_count=log_images(imgs,pred,0,self.experiment,self.args.ckptpath) #image_upload_count
       
    
    ###8. save  ###
    def save(self, epoch):
       "save model to file"
       self.saver.save(self.sess, join(self.args.ckptpath, 'model'), global_step=epoch)


In [5]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        #arprint=ArtPrint(args.data_root, transform=transform_train)
        arhh=ArtPrintNoIntsectLBW_biameyd_siameyd(args.data_root, transform=transform_train)
        arph=ArtPrintNoIntsectLBW(args.data_root, transform=transform_train)
        arpp=ArtPrintNoIntsectLBW_bpr_spr(args.data_root, transform=transform_train)
        arhp=ArtPrintNoIntsectLBW_biameyd_sprt(args.data_root, transform=transform_train)
        concat=ConcatDataset([arhh,arph,arpp,arhp])
        print(len(concat))
        idxTrain = int( len(concat)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        #return trainset, concat
        #weight gen
        # modified here:
        samples_all=[]
        print(trainset.dataset.datasets)
        for ds in trainset.dataset.datasets:
            samples_all+=ds.samples
        #samples_all=trainset.dataset.samples
        print(len(samples_all))
        print(samples_all[0])
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),samples_all))/(args.image_h*args.image_w*len(samples_all))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    elif:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        irs = IRS(args.dataroot,transform=transform_train) #yike todo
        testloader = DataLoader(irs, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        model=Model(args, experiment, loss_weight=[0.5,0.5], mustRestore=False)
        for idx, (images, labels) in enumerate(loader):
            images=images.numpy()
            model.imageClean(images)
    else:
        
        pass # for now, leave it pass 
    